In [11]:
#-*-coding:utf-8-*-
import os
import random # Analysis
import datetime as dt # Analysis
import numpy as np # Analysis
import pandas as pd # Analysis
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

import warnings
warnings.filterwarnings(action='ignore') # ignore warnings
# datafarme 보여주는 범위 설정

pd.options.display.max_columns=1000
pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format


df = {}

for i in range(2010, 2022):
    df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
    df[i].drop(labels=df[i][df[i]['아내지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내직업코드'] == 99].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편직업코드'] == 99].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편교육코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내교육코드'] == 9].index, inplace=True)
for i in range(2010, 2021):
    df[i].drop(labels=df[i][df[i]['남편국적구분코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내국적구분코드'] == 9].index, inplace=True)

mandic = {1:10015,2:5011,3:4833,5:3325.5,6:2582,7:3510,8:3539,9:2494,13:0}
womandic = {1:8024,2:3136,3:3256,5:2119,6:2060,7:2156,8:2795,9:1897,13:0}

for i in range(2010,2022):
    df[i]['남편소득수준'] = df[i]['남편직업코드']
    df[i]['아내소득수준'] = df[i]['아내직업코드']
    
    for j in mandic.keys():
        df[i]['남편소득수준'].replace(j,mandic[j], inplace=True)
        df[i]['아내소득수준'].replace(j,mandic[j], inplace=True)
        
#기본전처리. 
#연도삭제
plt.rcParams['font.family'] = 'HYGraphic-Medium' # 깨짐방지
#교육수준 통합 필요.


# 3 -> 중졸 이히
# 4 -> 고졸
# 5 -> 대졸
# 6 -> 대학원졸
for i in range(2010, 2022):
    df[i].drop(df[i][df[i]['남편교육코드'] >= 7].index, inplace=True)
    df[i].loc[df[i]["남편교육코드"] == 1, "남편교육코드"] = 3
    df[i].loc[df[i]["남편교육코드"] == 2, "남편교육코드"] = 3

In [12]:
dic = {1:0.038,2:0.123,3:0.2,4:0.285,5:0.367,
6:0.452,7:0.534,8:0.619,9:0.704,10:0.786,11:0.871,12:0.953}
for i in range(2010,2022):
    for j in dic.keys():
        df[i].loc[df[i]['실제결혼월'] == j,'남편출생년도'] = ((df[i]['실제결혼연도']+dic[j])-df[i]['남편연령']).apply(int)
        df[i].loc[df[i]['실제결혼월'] == j,'아내출생년도'] = ((df[i]['실제결혼연도']+dic[j])-df[i]['아내연령']).apply(int)

In [14]:
for year in range(2010,2022):
    # 소득  (연속)
    # 소득 차이 (연속)
    # 여성결혼연령 (연속)
    # 연상혼
    df[year].loc[round(df[year]['남편연령']) < round(df[year]['아내연령']), "강혼여부이전기준"] = 1
    df[year].loc[round(df[year]['남편연령']) >= round(df[year]['아내연령']), "강혼여부이전기준"] = 0
    df[year].loc[df[year]['남편출생년도'] > df[year]['아내출생년도'], "강혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] <= df[year]['아내출생년도'], "강혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] == df[year]['아내출생년도'], "동갑혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] != df[year]['아내출생년도'], "동갑혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] < df[year]['아내출생년도'], "연하혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] >= df[year]['아내출생년도'], "연하혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] > df[year]['아내출생년도'], "연상혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] <= df[year]['아내출생년도'], "연상혼여부"] = 0

    # 아내 대도시
    df[year].loc[df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), 
    "아내대도시여부"] = 1
    df[year].loc[~df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), 
    "아내대도시여부"] = 0

    # 남편 대도시
    df[year].loc[df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]),
    "남편대도시여부"] = 1
    df[year].loc[~df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]),
    "남편대도시여부"] = 0
    df[year].loc[df[year]['남편지역코드'].isin([11,23,31]),
    "남편수도권여부"] = 1
    df[year].loc[~df[year]['남편지역코드'].isin([11,23,31]),
    "남편수도권여부"] = 0
    df[year].loc[df[year]['아내지역코드'].isin([11,23,31]),
    "아내수도권여부"] = 1
    df[year].loc[~df[year]['아내지역코드'].isin([11,23,31]),
    "아내수도권여부"] = 0
    # 남자 결혼 종류

    # 여자 결혼 종류

    # 아내 재혼 여부
    df[year].loc[df[year]['아내결혼코드'] == 1,
    "아내재혼여부"] = 0
    df[year].loc[df[year]['아내결혼코드'] != 1,
    "아내재혼여부"]= 1

    # 남편 재혼 여부
    df[year].loc[df[year]['남편결혼코드'] == 1, 
    "남편재혼여부"] = 0
    df[year].loc[df[year]['남편결혼코드'] != 1, 
    "남편재혼여부"]= 1

    # 결혼 종류 차이 여부
    df[year].loc[df[year]['남편재혼여부'] == df[year]['아내재혼여부'], 
    "결혼종류차이여부"] = 1
    df[year].loc[df[year]['남편재혼여부'] != df[year]['아내재혼여부'],
    "결혼종류차이여부"]= 0

    # 강혼여부 여부
    df[year].loc[df[year]['아내교육코드'] > df[year]['남편교육코드'],
    "강혼여부"] = 1
    df[year].loc[~(df[year]['아내교육코드'] > df[year]['남편교육코드']),
    "강혼여부"] = 0

    # 동질혼 여부
    df[year].loc[df[year]['아내교육코드'] == df[year]['남편교육코드'],
    "동질혼여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != df[year]['남편교육코드'],
    "동질혼여부"] = 0

    # 승혼 여부
    df[year].loc[df[year]['아내교육코드'] < df[year]['남편교육코드'],
    "승혼여부"] = 1
    df[year].loc[~(df[year]['아내교육코드'] < df[year]['남편교육코드']),
    "승혼여부"] = 0

    # 소득 차
    df[year]['소득차'] = df[year]['아내소득수준'] - df[year]['남편소득수준']

    # 소득차 카테고리화
    df[year].loc[df[year]['소득차'] > 0, '소득차범주'] = 1
    df[year].loc[df[year]['소득차'] == 0, '소득차범주'] = 2
    df[year].loc[df[year]['소득차'] < 0, '소득차범주'] = 3

    df[year]['아내연령제곱'] = df[year]['아내연령'] ** 2
    df[year]['남편연령제곱'] = df[year]['남편연령'] ** 2

    # 아내 무직 여부
    df[year].loc[df[year]['아내직업코드'] == 13, "아내무직여부"] = 1
    df[year].loc[df[year]['아내직업코드'] != 13, "아내무직여부"] = 0

    # 남편 무직 여부
    df[year].loc[df[year]['남편직업코드'] == 13, "남편무직여부"] = 1
    df[year].loc[df[year]['남편직업코드'] != 13, "남편무직여부"] = 0

    df[year].loc[df[year]['남편지역코드'] == df[year]['아내지역코드'], "동일지역여부"] = 1
    df[year].loc[df[year]['남편지역코드'] != df[year]['아내지역코드'], "동일지역여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 3, "아내중졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != 3, "아내중졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 3, "남편중졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] != 3, "남편중졸여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 4, "아내고졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != 4, "아내고졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 4, "남편고졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] != 4, "남편고졸여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 5, "아내대졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] !=5, "아내대졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 5, "남편대졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=5, "남편대졸여부"] = 0

    df[year].loc[df[year]['남편교육코드'] == 6, "남편대학원졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=6, "남편대학원졸여부"] = 0
    df[year].loc[df[year]['아내교육코드'] == 6, "아내대학원졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] !=6, "아내대학원졸여부"] = 0

    df[year].loc[df[year]['남편교육코드'] ==df[year]['아내교육코드'] , "동질혼여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=df[year]['아내교육코드'] , "동질혼여부"] = 0
    

for year in range(2010,2022):
    df[year]['아내국제결혼여부'] = 0
    df[year].loc[((df[year]['아내국적구분코드']>= 2)&(df[year]['신고연도']!=2021)), "아내국제결혼여부"] = 1
    df[year]['남편국제결혼여부'] = 0
    df[year].loc[((df[year]['남편국적구분코드']>=2)&(df[year]['신고연도']!=2021)), "남편국제결혼여부"] = 1
    df[year].loc[((df[year]['아내지역코드']>80)&(df[year]['신고연도'] == 2021)), "아내국제결혼여부"] = 1
    df[year].loc[(~(df[year]['아내지역코드']>80)&(df[year]['신고연도'] == 2021)), "아내국제결혼여부"] = 0
    df[year].loc[((df[year]['남편지역코드']>80)&(df[year]['신고연도'] == 2021)), "남편국제결혼여부"] = 1
    df[year].loc[(~(df[year]['남편지역코드']>80)&(df[year]['신고연도'] == 2021)), "남편국제결혼여부"] = 0

In [15]:
df_from_2014_to_2021 = df[2021].append(df[2020]).append(df[2019]).append(df[2018]).append(df[2017]).append(df[2016]).append(df[2015]).append(df[2014])

In [16]:
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2016)|(df_from_2014_to_2021['실제결혼연도'] == 2017), "1617"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2018)|(df_from_2014_to_2021['실제결혼연도'] == 2019), "1819"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2020)|(df_from_2014_to_2021['실제결혼연도'] == 2021), "2021"] = 1
df_from_2014_to_2021.loc[~((df_from_2014_to_2021['실제결혼연도'] == 2016)|(df_from_2014_to_2021['실제결혼연도'] == 2017)), "1617"] = 0
df_from_2014_to_2021.loc[~((df_from_2014_to_2021['실제결혼연도'] == 2018)|(df_from_2014_to_2021['실제결혼연도'] == 2019)), "1819"] = 0
df_from_2014_to_2021.loc[~((df_from_2014_to_2021['실제결혼연도'] == 2020)|(df_from_2014_to_2021['실제결혼연도'] == 2021)), "2021"] = 0

In [25]:
l = ['아내연령','아내재혼여부','남편재혼여부','아내국제결혼여부',
'남편국제결혼여부','아내중졸여부','아내고졸여부','아내대학원졸여부','동일지역여부',
'소득차','1617','1819','2021', '강혼여부', '연상혼여부']

d = pd.DataFrame()

d.index = np.array(l)

for index in l:
    d.loc[index, '평균'] = df_from_2014_to_2021[index].mean()
    d.loc[index, '표준편차'] = df_from_2014_to_2021[index].std()
    d.loc[index, '최소값'] = df_from_2014_to_2021[index].min()
    d.loc[index, '최대값'] = df_from_2014_to_2021[index].max()

d




,평균,표준편차,최소값,최대값
아내연령,32.60608,7.97335,15.07300,95.47600
아내재혼여부,0.16126,0.36777,0.00000,1.00000
남편재혼여부,0.14094,0.34796,0.00000,1.00000
아내국제결혼여부,0.05978,0.23708,0.00000,1.00000
남편국제결혼여부,0.02012,0.14042,0.00000,1.00000
아내중졸여부,0.03787,0.19089,0.00000,1.00000
아내고졸여부,0.24461,0.42985,0.00000,1.00000
아내대학원졸여부,0.07325,0.26055,0.00000,1.00000
동일지역여부,0.72364,0.44720,0.00000,1.00000
소득차,-1274.79147,2759.82419,-10015.00000,10015.00000
